In [3]:
import pandas as pd
import csv
import random

In [4]:
csv.field_size_limit(10**6*10)

131072

In [5]:
# load a df from cleaned_normalized_df_no_grouping.pkl
df = pd.read_pickle('cleaned_normalized_df_no_grouping.pkl')

In [6]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get('API_KEY')
api_url = os.environ.get('API_URL')
url = api_url + '/index_document'

class IndexDocumentRequest:
    def __init__(self, doc_html, story_id, index):
        self.doc_html = doc_html
        self.story_id = story_id
        self.index = index

    def get_payload(self):
        json_payload = {
            'doc_html': self.doc_html,
            'story_id': self.story_id,
            'index': self.index
        }
        return json.dumps(json_payload)
    
    def send_post_request(self):
        stringified_json_payload = self.get_payload()
        headers = {"Content-Type": "application/json", "Authorization":api_key}
        req_result = requests.post(url, data=stringified_json_payload, headers=headers)

        if req_result.status_code != 200:
            req_error = req_result.text
            print(req_error)

In [12]:
def create_random_chunks():
    num_rows = df.shape[0]

    # Randomly select 5 row indices
    random_indice = random.sample(range(0, num_rows), 1)
    random_range = []
    for i in range(0, 5):
        random_range.append(random_indice[0] + i)

    # Sort the DataFrame based on the selected indices
    sampled_rows = df.iloc[random_range]

    # get the first 10 rows with a FictionId of 24
    rows_24 = df.loc[df['FictionId'] == 24].head(10)

    for index, row in rows_24.iterrows():
        doc_html = row['content']
        story_id = int(row['FictionId'])
        index = int(row['Order'])
        index_doc_request = IndexDocumentRequest(doc_html, story_id, index)
        index_doc_request.send_post_request()

In [13]:
for i in range(0, 10):
    create_random_chunks()

{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}


In [16]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333", api_key="qdrant_pass")

client.count(collection_name="doc_group_5")

CountResult(count=2)